In [1]:
import pandas as pd
import numpy as np
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import re
import ast 
import json

In [112]:
quality = pd.read_csv('./oecd_quality_uspto_jan20.csv')

In [3]:
info_patents = pd.read_pickle('./info_patents')

In [113]:
quality.columns

Index(['appln_id', 'pub_nbr', 'filing', 'tech_field', 'many_field',
       'patent_scope', 'family_size', 'grant_lag', 'bwd_cits', 'npl_cits',
       'claims', 'claims_bwd', 'fwd_cits5', 'fwd_cits7', 'breakthrough',
       'generality', 'originality', 'radicalness', 'renewal',
       'quality_index_4', 'quality_index_6'],
      dtype='object')

In [114]:
quality

,appln_id,pub_nbr,filing,tech_field,many_field,patent_scope,family_size,grant_lag,bwd_cits,npl_cits,...,claims_bwd,fwd_cits5,fwd_cits7,breakthrough,generality,originality,radicalness,renewal,quality_index_4,quality_index_6
0,49683821,US03951277,1976,25,0,1,5,106.0,5,0,...,NaN,0,0,0,NaN,0.812500,0.875000,0.0,0.412698,0.480100
1,48696358,US03956062,1976,27,0,1,1,0.0,7,0,...,NaN,1,1,0,0.0,0.444444,1.000000,0.0,0.072070,0.249299
2,51805403,US03960276,1976,25,0,2,10,83.0,4,0,...,NaN,0,2,0,NaN,0.720000,1.000000,0.0,0.246449,0.367851
3,51041941,US03963460,1976,24,1,3,7,72.0,5,0,...,NaN,0,1,0,NaN,0.728889,0.466667,0.0,0.268662,0.387728
4,51841776,US03963546,1976,26,0,2,1,82.0,6,0,...,NaN,0,1,0,NaN,0.743802,0.727273,0.0,0.113912,0.316085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8574380,516842427,US2019230828,2019,1,1,2,1,NaN,0,0,...,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
8574381,516842321,US2019230829,2016,2,0,1,4,NaN,0,0,...,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
8574382,516842367,US2019230830,2016,2,0,1,4,NaN,0,0,...,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
8574383,516851404,US2019230831,2016,25,1,2,4,NaN,0,0,...,NaN,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [115]:
np.unique([len(quality.pub_nbr[i]) for i in range(len(quality))],return_counts=True)

(array([10, 12, 13]), array([6270188, 2304196,       1]))

In [10]:
test[test.pub_nbr=='US05387603']

,appln_id,pub_nbr,filing,tech_field,many_field,patent_scope,family_size,grant_lag,bwd_cits,npl_cits,...,claims_bwd,fwd_cits5,fwd_cits7,breakthrough,generality,originality,radicalness,renewal,quality_index_4,quality_index_6
1336214,48266630,US05387603,1993,16,1,3,6,433.0,4,0,...,NaN,0,1,0,NaN,0.922495,0.086957,13.0,0.141889,0.270958


In [38]:
info_patents.patent_number = pd.to_numeric(info_patents.patent_number,errors='coerce')

In [40]:
info_patents = info_patents.dropna()

/Users/gregoirebellan/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [4]:
info_patents

,patent_number,patent_date,patent_processing_time,ipc_section,assignee_id,wipos,marked
0,5387603,1995-02-07,433,"[A, C]",[5330279e-a21e-438c-80d1-7e6ae793c092],"[16, 14]",1.0
1,5771295,1998-06-23,552,[H],[2986e46e-7937-477f-9bc5-1f5eec05183f],[2],1.0
2,5585587,1996-12-17,832,[G],[f7152515-d420-42d9-a7b6-3a2796bc5d10],"[34, 2, 10]",1.0
3,5321490,1994-06-14,568,[G],[ec72750e-f789-4749-91f7-6292f7c6fda7],"[10, 12, 29]",1.0
4,5750497,1998-05-12,1161,"[A, C]",[9a3afa40-66e5-4921-b831-64287f2052e0],"[16, 15]",1.0
...,...,...,...,...,...,...,...
1915706,D555179,2007-11-13,299,None,[dc77a1b1-f087-4a19-992d-6393caf66347],None,0.0
1915707,4192319,1980-03-11,893,[A],[fa543ac8-5ee8-4999-b949-6c8a3dbae65d],[13],0.0
1915708,8386739,2013-02-26,1247,[G],[ee0168d9-bf74-4513-bfbf-e73c933f90e7],[6],0.0
1915709,4191891,1980-03-04,481,[H],[f2d3dc32-b29c-4c55-ab6d-736b4ee8b96b],[13],0.0


In [6]:
info_patents.patent_processing_time.astype(int)

ValueError: invalid literal for int() with base 10: '1970-01-01 00:13:03'

In [7]:
info_patents = info_patents.dropna()
info_patents = info_patents[info_patents.patent_date >= '2000-01-01']
info_patents = info_patents[info_patents.patent_date <= '2019-01-01']
info_patents = info_patents.reset_index(drop=True)

In [21]:
info_patents = info_patents[info_patents.patent_processing_time.apply(lambda x: str(x).isdigit())].reset_index(drop=True)
info_patents.patent_processing_time = info_patents.patent_processing_time.astype(int)

In [39]:
info_patents

,patent_number,patent_date,patent_processing_time,ipc_section,assignee_id,wipos,marked
0,6127678,2000-10-03,329,[H],[019b9aba-95cd-43bf-a8a5-e700229fddac],"[28, 25]",1.0
1,6073289,2000-06-13,908,[A],[e5fc32c3-a913-455a-94ce-9932d30eefff],[13],1.0
2,6073290,2000-06-13,944,[A],[3860ed0c-8e84-4bed-ba0b-9d7a57d3a6b9],"[29, 13]",1.0
3,6064927,2000-05-16,995,"[B, G]",[996d1179-2d0d-4a05-ac9f-32c3eb34fb69],"[10, 32]",1.0
4,6029495,2000-02-29,651,[G],[0f63d31a-dac4-4326-afe8-e0d9ca77e826],[10],1.0
...,...,...,...,...,...,...,...
903441,8386963,2013-02-26,1370,[G],[4674eaf4-2927-46ab-ae18-9d92bfcce864],[6],0.0
903442,8386259,2013-02-26,2252,[G],[1b7df99f-bb3a-4026-a4ab-f5f89558b405],"[6, 3, 4]",0.0
903443,8386822,2013-02-26,1852,[G],[ee0168d9-bf74-4513-bfbf-e73c933f90e7],[6],0.0
903444,8386739,2013-02-26,1247,[G],[ee0168d9-bf74-4513-bfbf-e73c933f90e7],[6],0.0


In [55]:
test = info_patents.append([info_patents]*18)

In [56]:
test

,patent_number,patent_date,patent_processing_time,ipc_section,assignee_id,wipos,marked
0,6127678,2000-10-03,329,[H],[019b9aba-95cd-43bf-a8a5-e700229fddac],"[28, 25]",1.0
1,6073289,2000-06-13,908,[A],[e5fc32c3-a913-455a-94ce-9932d30eefff],[13],1.0
2,6073290,2000-06-13,944,[A],[3860ed0c-8e84-4bed-ba0b-9d7a57d3a6b9],"[29, 13]",1.0
3,6064927,2000-05-16,995,"[B, G]",[996d1179-2d0d-4a05-ac9f-32c3eb34fb69],"[10, 32]",1.0
4,6029495,2000-02-29,651,[G],[0f63d31a-dac4-4326-afe8-e0d9ca77e826],[10],1.0
...,...,...,...,...,...,...,...
903441,8386963,2013-02-26,1370,[G],[4674eaf4-2927-46ab-ae18-9d92bfcce864],[6],0.0
903442,8386259,2013-02-26,2252,[G],[1b7df99f-bb3a-4026-a4ab-f5f89558b405],"[6, 3, 4]",0.0
903443,8386822,2013-02-26,1852,[G],[ee0168d9-bf74-4513-bfbf-e73c933f90e7],[6],0.0
903444,8386739,2013-02-26,1247,[G],[ee0168d9-bf74-4513-bfbf-e73c933f90e7],[6],0.0


In [57]:
test = test.sort_values(by='patent_number',ascending='True')

In [58]:
test

,patent_number,patent_date,patent_processing_time,ipc_section,assignee_id,wipos,marked
386384,10000007,2018-06-19,739,[B],[3441622b-13af-4ccb-a658-dd9fb2f9c25b],[29],0.0
386384,10000007,2018-06-19,739,[B],[3441622b-13af-4ccb-a658-dd9fb2f9c25b],[29],0.0
386384,10000007,2018-06-19,739,[B],[3441622b-13af-4ccb-a658-dd9fb2f9c25b],[29],0.0
386384,10000007,2018-06-19,739,[B],[3441622b-13af-4ccb-a658-dd9fb2f9c25b],[29],0.0
386384,10000007,2018-06-19,739,[B],[3441622b-13af-4ccb-a658-dd9fb2f9c25b],[29],0.0
...,...,...,...,...,...,...,...
386413,9999989,2018-06-19,1922,[B],[c8db19e0-0ff3-42f0-bf3a-4f8ce43d7e2c],"[21, 29]",0.0
386413,9999989,2018-06-19,1922,[B],[c8db19e0-0ff3-42f0-bf3a-4f8ce43d7e2c],"[21, 29]",0.0
386413,9999989,2018-06-19,1922,[B],[c8db19e0-0ff3-42f0-bf3a-4f8ce43d7e2c],"[21, 29]",0.0
386413,9999989,2018-06-19,1922,[B],[c8db19e0-0ff3-42f0-bf3a-4f8ce43d7e2c],"[21, 29]",0.0


In [59]:
test["Year"] = [2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018]*len(info_patents)

In [62]:
test = test.set_index(['patent_number','Year'])
test.reset_index(level=['Year'], inplace=True)

In [74]:
test['granted']=pd.DatetimeIndex(test.patent_date).year

In [93]:
test.granted=test.granted.apply(lambda x: x==Year)

NameError: name 'Year' is not defined

In [98]:
test.granted = test.granted==test.Year

In [100]:
test.head(20)

,Year,patent_date,patent_processing_time,ipc_section,assignee_id,wipos,marked,granted
patent_number,,,,,,,,
10000007,2000,2018-06-19,739,[B],[3441622b-13af-4ccb-a658-dd9fb2f9c25b],[29],0.0,False
10000007,2001,2018-06-19,739,[B],[3441622b-13af-4ccb-a658-dd9fb2f9c25b],[29],0.0,False
10000007,2002,2018-06-19,739,[B],[3441622b-13af-4ccb-a658-dd9fb2f9c25b],[29],0.0,False
10000007,2003,2018-06-19,739,[B],[3441622b-13af-4ccb-a658-dd9fb2f9c25b],[29],0.0,False
10000007,2004,2018-06-19,739,[B],[3441622b-13af-4ccb-a658-dd9fb2f9c25b],[29],0.0,False
10000007,2005,2018-06-19,739,[B],[3441622b-13af-4ccb-a658-dd9fb2f9c25b],[29],0.0,False
10000007,2006,2018-06-19,739,[B],[3441622b-13af-4ccb-a658-dd9fb2f9c25b],[29],0.0,False
10000007,2007,2018-06-19,739,[B],[3441622b-13af-4ccb-a658-dd9fb2f9c25b],[29],0.0,False
10000007,2008,2018-06-19,739,[B],[3441622b-13af-4ccb-a658-dd9fb2f9c25b],[29],0.0,False


In [111]:
np.unique([len(info_patents.patent_number[i]) for i in range(0,903446)],return_counts=True)

(array([7, 8]), array([867379,  36067]))

In [122]:
quality = quality[quality.pub_nbr.apply(lambda x: len(x)==10)]

In [123]:
quality.pub_nbr = quality.pub_nbr.apply(lambda x: x==x[2:])

,appln_id,pub_nbr,filing,tech_field,many_field,patent_scope,family_size,grant_lag,bwd_cits,npl_cits,...,claims_bwd,fwd_cits5,fwd_cits7,breakthrough,generality,originality,radicalness,renewal,quality_index_4,quality_index_6
0,49683821,US03951277,1976,25,0,1,5,106.0,5,0,...,NaN,0,0,0,NaN,0.812500,0.875000,0.0,0.412698,0.480100
1,48696358,US03956062,1976,27,0,1,1,0.0,7,0,...,NaN,1,1,0,0.0,0.444444,1.000000,0.0,0.072070,0.249299
2,51805403,US03960276,1976,25,0,2,10,83.0,4,0,...,NaN,0,2,0,NaN,0.720000,1.000000,0.0,0.246449,0.367851
3,51041941,US03963460,1976,24,1,3,7,72.0,5,0,...,NaN,0,1,0,NaN,0.728889,0.466667,0.0,0.268662,0.387728
4,51841776,US03963546,1976,26,0,2,1,82.0,6,0,...,NaN,0,1,0,NaN,0.743802,0.727273,0.0,0.113912,0.316085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6270183,477846335,US10362716,2015,2,1,2,1,1369.0,0,0,...,NaN,0,0,0,NaN,NaN,NaN,4.0,0.094294,0.075252
6270184,498406497,US10362717,2016,21,1,2,4,1070.0,0,0,...,NaN,0,0,0,NaN,NaN,NaN,3.0,0.273504,0.191243
6270185,496339505,US10362718,2017,1,1,3,3,592.0,0,0,...,NaN,1,1,0,0.5,NaN,NaN,2.0,0.224706,0.203709
6270186,479265472,US10362719,2014,2,0,1,4,1938.0,6,0,...,NaN,0,0,0,NaN,0.617284,0.222222,5.0,0.128205,0.086840


In [132]:
quality.pub_nbr = quality.pub_nbr.str.strip('US')

AttributeError: Can only use .str accessor with string values!

In [ ]:
quality.pub_nbr = quality.pub_nbr.astype(str)

In [138]:
quality = quality[quality.pub_nbr.isin(info_patents.patent_number)]

In [139]:
quality

,appln_id,pub_nbr,filing,tech_field,many_field,patent_scope,family_size,grant_lag,bwd_cits,npl_cits,...,claims_bwd,fwd_cits5,fwd_cits7,breakthrough,generality,originality,radicalness,renewal,quality_index_4,quality_index_6
1949720,53758551,6009558,1997,13,1,2,11,1001.0,20,0,...,NaN,13,28,0,0.717031,0.883654,0.542373,14.0,0.345639,0.352320
1949726,47536803,6009564,1998,9,1,2,1,559.0,50,4,...,NaN,8,22,0,0.642361,0.886656,0.352000,13.0,0.291047,0.426648
1949742,54472680,6009580,1997,13,1,2,9,743.0,7,0,...,NaN,11,11,0,0.662688,0.824074,0.250000,14.0,0.347508,0.352031
1949750,47750230,6009588,1998,23,1,2,8,537.0,13,0,...,NaN,2,6,0,0.375000,0.588235,0.117647,13.0,0.380807,0.397684
1949751,51779540,6009589,1995,34,0,1,10,1512.0,12,1,...,NaN,11,13,0,0.561983,0.870392,0.482759,8.0,0.499056,0.402245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6081979,499693198,10172250,2017,2,1,3,1,390.0,42,3,...,NaN,0,0,0,NaN,0.879106,0.699187,2.0,0.120879,0.215613
6081982,500076233,10172253,2018,2,0,2,1,231.0,111,26,...,NaN,0,0,0,NaN,0.966187,0.575145,1.0,0.199248,0.433835
6081984,497510490,10172255,2017,32,1,2,1,705.0,17,3,...,NaN,0,0,0,NaN,0.897234,0.590476,2.0,0.193900,0.177196
6081987,481029029,10172258,2017,8,1,3,1,655.0,25,5,...,NaN,0,0,0,NaN,0.833986,0.325843,2.0,0.148629,0.159460
